In [ ]:
pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 919.9 kB/s eta 0:00:00


In [ ]:
import tensorflow_datasets as tfds
import mido
import io
import numpy as np

import tensorflow_datasets as tfds

# Load and shuffle the dataset
dataset = tfds.load(name="groove/full-midionly", split="train", try_gcs=True).shuffle(buffer_size=10000)

# Take a subset
subset_size = 100  # for example
dataset_subset = dataset.take(subset_size)

# Now proceed with your existing processing code on `dataset_subset`

# Define the process_midi_data function
def process_midi_data(midi_info):
    genre = midi_info['genre'].numpy()  # Assuming genre is a TensorFlow tensor
    tempo = midi_info['tempo']
    time_signature = midi_info['time_signature']
    note_list = midi_info['notes']

    processed_notes = []
    for note in note_list:
        note_number = note['note']
        velocity = note['velocity']
        time = note['time']
        processed_note = [note_number, velocity, time]
        processed_notes.append(processed_note)

    processed_notes_array = np.array(processed_notes)
    return genre, tempo, time_signature, processed_notes_array


# Initialize a list to store the processed data
processed_data = []

# Iterate through the dataset
for features in dataset_subset:
    midi_data = features['midi']
    genre = features["style"]["primary"]
    midi_bytes = midi_data.numpy()
    midi_stream = io.BytesIO(midi_bytes)

    try:
        midi_file = mido.MidiFile(file=midi_stream)
        midi_info = {"genre": genre, "tempo": None, "time_signature": None, "notes": []}

        for track in midi_file.tracks:
            for msg in track:
                if msg.is_meta:
                    if msg.type == 'set_tempo':
                        tempo = mido.tempo2bpm(msg.tempo)
                        midi_info["tempo"] = tempo
                    elif msg.type == 'time_signature':
                        time_signature = f"{msg.numerator}/{msg.denominator}"
                        midi_info["time_signature"] = time_signature
                else:
                    if msg.type in ['note_on', 'note_off']:
                        midi_info["notes"].append({"note": msg.note, "velocity": msg.velocity, "time": msg.time})

        # Process the extracted MIDI data
        processed_midi_data = process_midi_data(midi_info)
        processed_data.append(processed_midi_data)

    except Exception as e:
        print(f"Error reading MIDI file: {e}")

# Now, processed_data contains the structured data from each MIDI file


decide on a sequence length for your LSTM inputs. This means how many notes the network looks at to predict the next note.Also, scale the data.

In [ ]:
import tensorflow_datasets as tfds
import mido
import io
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define sequence length and batch size
sequence_length = 50
batch_size = 10  # Process 100 MIDI files at a time

def process_batch(dataset_batch):
    input_sequences = []
    output_notes = []

    # `dataset_batch` is a batch of features
    midi_data_batch = dataset_batch['midi']

    for midi_data in midi_data_batch:
        midi_bytes = midi_data.numpy()
        midi_stream = io.BytesIO(midi_bytes)

        try:
            midi_file = mido.MidiFile(file=midi_stream)
            notes = []
            for track in midi_file.tracks:
                for msg in track:
                    if msg.type in ['note_on', 'note_off']:
                        notes.append([msg.note, msg.velocity, msg.time])

            # Create sequences for each MIDI file
            for i in range(0, len(notes) - sequence_length):
                input_seq = notes[i:i + sequence_length]
                output_note = notes[i + sequence_length]
                input_sequences.append(input_seq)
                output_notes.append(output_note)

        except Exception as e:
            print(f"Error reading MIDI file: {e}")

    return input_sequences, output_notes


# Load the dataset
#dataset = tfds.load(name="groove/full-midionly", split=tfds.Split.TRAIN, try_gcs=True)

# Process the dataset in batches
all_input_sequences = []
all_output_notes = []

for batch in dataset_subset.batch(batch_size):
    input_sequences, output_notes = process_batch(batch)
    all_input_sequences += input_sequences
    all_output_notes += output_notes


# Find the min and max for note, velocity, and time
min_note = min([note[0] for seq in all_input_sequences for note in seq])
max_note = max([note[0] for seq in all_input_sequences for note in seq])
min_velocity = min([note[1] for seq in all_input_sequences for note in seq])
max_velocity = max([note[1] for seq in all_input_sequences for note in seq])
min_time = min([note[2] for seq in all_input_sequences for note in seq])
max_time = max([note[2] for seq in all_input_sequences for note in seq])
'''
# Fit the scaler for each
note_scaler = MinMaxScaler(feature_range=(0, 1)).fit([[min_note], [max_note]])
velocity_scaler = MinMaxScaler(feature_range=(0, 1)).fit([[min_velocity], [max_velocity]])
time_scaler = MinMaxScaler(feature_range=(0, 1)).fit([[min_time], [max_time]])

'''

'''
# Prepare the entire dataset for fitting the scalers
all_notes = [note[0] for seq in all_input_sequences for note in seq]
all_velocities = [note[1] for seq in all_input_sequences for note in seq]
all_times = [note[2] for seq in all_input_sequences for note in seq]

# Fit the scalers with the entire dataset
note_scaler = MinMaxScaler(feature_range=(0, 1)).fit(np.array(all_notes).reshape(-1, 1))
velocity_scaler = MinMaxScaler(feature_range=(0, 1)).fit(np.array(all_velocities).reshape(-1, 1))
time_scaler = MinMaxScaler(feature_range=(0, 1)).fit(np.array(all_times).reshape(-1, 1))
'''

def simple_normalize(data, max_value):
    return data / max_value

def simple_denormalize(data, max_value):
    return (data * max_value).astype(int)


In [ ]:
all_notes = [note[0] for sequence in all_input_sequences for note in sequence]
all_velocities = [note[1] for sequence in all_input_sequences for note in sequence]
all_times = [note[2] for sequence in all_input_sequences for note in sequence]


normalized_notes = simple_normalize(np.array(all_notes), 127)  # Normalize notes
normalized_velocities = simple_normalize(np.array(all_velocities), 127)  # Normalize velocities

# For time, determine an appropriate max value based on your data
max_time_value = max(all_times)  # or a predefined value if you have one
normalized_times = simple_normalize(np.array(all_times), max_time_value)  # Normalize times


normalized_dataset = []
for i in range(0, len(all_notes), sequence_length):
    sequence = [[normalized_notes[i + j], normalized_velocities[i + j], normalized_times[i + j]]
                for j in range(sequence_length)]
    normalized_dataset.append(sequence)

normalized_dataset = np.array(normalized_dataset)



In [ ]:
print(normalized_dataset.shape)
print(normalized_dataset)

(72984, 50, 3)
[[[4.33070866e-01 4.17322835e-01 0.00000000e+00]
  [4.01574803e-01 1.00000000e+00 3.82304752e-03]
  [2.83464567e-01 3.77952756e-01 4.91534681e-03]
  ...
  [4.01574803e-01 0.00000000e+00 0.00000000e+00]
  [3.46456693e-01 7.48031496e-01 5.46149645e-04]
  [4.01574803e-01 7.71653543e-01 8.19224468e-03]]

 [[4.01574803e-01 1.00000000e+00 3.82304752e-03]
  [2.83464567e-01 3.77952756e-01 4.91534681e-03]
  [4.33070866e-01 0.00000000e+00 4.64227198e-02]
  ...
  [3.46456693e-01 7.48031496e-01 5.46149645e-04]
  [4.01574803e-01 7.71653543e-01 8.19224468e-03]
  [3.46456693e-01 0.00000000e+00 2.83997815e-02]]

 [[2.83464567e-01 3.77952756e-01 4.91534681e-03]
  [4.33070866e-01 0.00000000e+00 4.64227198e-02]
  [4.01574803e-01 0.00000000e+00 3.82304752e-03]
  ...
  [4.01574803e-01 7.71653543e-01 8.19224468e-03]
  [3.46456693e-01 0.00000000e+00 2.83997815e-02]
  [4.01574803e-01 0.00000000e+00 8.19224468e-03]]

 ...

 [[3.70078740e-01 3.22834646e-01 1.85690879e-02]
  [3.38582677e-01 5.0393

In [ ]:
'''
# Normalize the input sequences
normalized_input_sequences = np.array([
    [[note_scaler.transform([[note[0]]])[0][0], velocity_scaler.transform([[note[1]]])[0][0], time_scaler.transform([[note[2]]])[0][0]]
     for note in seq] for seq in all_input_sequences
])
'''

In [ ]:
'''
print(normalized_input_sequences.shape)
'''

(66442, 50, 3)


In [ ]:
# Initialize the output array with the appropriate shape
num_unique_notes = len(note_to_int)
one_hot_encoded_output = np.zeros((len(all_output_notes), num_unique_notes + 2))  # +2 for velocity and time

# Define the maximum value for time normalization
max_time_value = max([note[2] for note in all_output_notes])  # Or set a predefined maximum time value

for i, note in enumerate(all_output_notes):
    # One-hot encode the note
    note_index = note_to_int[note[0]]
    one_hot_encoded_output[i, note_index] = 1

    # Simple normalization for velocity and time
    normalized_velocity = simple_normalize(note[1], 127)
    normalized_time = simple_normalize(note[2], max_time_value)

    # Store the normalized values
    one_hot_encoded_output[i, -2] = normalized_velocity  # Normalized velocity
    one_hot_encoded_output[i, -1] = normalized_time     # Normalized time


NameError: name 'note_to_int' is not defined

In [ ]:
print(one_hot_encoded_output.shape)
print(one_hot_encoded_output)

NameError: name 'one_hot_encoded_output' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np

# Save the normalized input sequences
np.save('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy', normalized_dataset)

# Save the one-hot encoded output
np.save('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/one_hot_encoded_output.npy', one_hot_encoded_output)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy'

In [ ]:

import tensorflow_datasets as tfds
import mido
import io
import numpy as np

import tensorflow_datasets as tfds
# Load the normalized input sequences
normalized_input_sequences = np.load('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy')

# Load the one-hot encoded output
one_hot_encoded_output = np.load('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/one_hot_encoded_output.npy')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy'

In [ ]:
'''
import numpy as np

# Save the normalized input sequences
np.save('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy', normalized_input_sequences)

# Save the one-hot encoded output
np.save('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/one_hot_encoded_output.npy', one_hot_encoded_output)
'''

In [ ]:
'''
import tensorflow_datasets as tfds
import mido
import io
import numpy as np

import tensorflow_datasets as tfds
# Load the normalized input sequences
normalized_input_sequences = np.load('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/normalized_input_sequences.npy')

# Load the one-hot encoded output
one_hot_encoded_output = np.load('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/one_hot_encoded_output.npy')
'''

In [ ]:
'''
print(normalized_input_sequences)
'''

In [ ]:
print(one_hot_encoded_output.shape)
#print(len(unique_notes))

(66136, 24)


In [ ]:
from sklearn.model_selection import train_test_split
# Split the data
X_train, X_test, y_train, y_test = train_test_split(normalized_dataset, one_hot_encoded_output, test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(52908, 50, 3)
(13228, 50, 3)
(52908, 24)


In [ ]:
print("NaNs in X_train:", np.isnan(X_train).any())
print("Infs in X_train:", np.isinf(X_train).any())
print("NaNs in y_train:", np.isnan(y_train).any())
print("Infs in y_train:", np.isinf(y_train).any())


NaNs in X_train: False
Infs in X_train: False
NaNs in y_train: False
Infs in y_train: False


In [ ]:
print(type(one_hot_encoded_output))
print(one_hot_encoded_output[0])
print(one_hot_encoded_output[1])



<class 'numpy.ndarray'>
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         0.         0.         0.69291339 0.00203943]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         0.         0.         0.50393701 0.05710401]


Define the Model Architecture:

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error

def custom_loss(y_true, y_pred):
    # Assuming the first part of y_pred is categorical (one-hot encoded notes) and the rest is continuous
    note_pred = y_pred[:, :22]
    continuous_pred = y_pred[:, 22:]#All 22s sould be replaced by len(unique_notes)

    note_true = y_true[:, :22]
    continuous_true = y_true[:, 22:]

    # Categorical loss (for notes)
    cat_loss = categorical_crossentropy(note_true, note_pred)

    # Mean squared error (for velocity and time)
    mse_loss = mean_squared_error(continuous_true, continuous_pred)

    # Combine the losses
    combined_loss = cat_loss + mse_loss

    return combined_loss


In [ ]:

'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(24, activation='softmax')) #24 should be len(unique_notes)+2
'''
from tensorflow.keras.layers import Dense, LSTM, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import Input

# Input layer
input_layer = Input(shape=(sequence_length, 3))  # 3 features: note, velocity, time

# LSTM layers
x = LSTM(256, return_sequences=True)(input_layer)
x = Dropout(0.3)(x)
x = LSTM(256, return_sequences=True)(x)
x = Dropout(0.3)(x)
x = LSTM(256)(x)
x = Dense(256)(x)
x = Dropout(0.3)(x)

# Separate output layers
output_note = Dense(len(unique_notes), activation='softmax', name='output_note')(x)
output_continuous = Dense(2, activation='linear', name='output_continuous')(x)  # 2 units for velocity and time

# Concatenate outputs
final_output = Concatenate()([output_note, output_continuous])

# Define the model
model = Model(inputs=input_layer, outputs=final_output)




Compile the Model:

In [ ]:
# Compile the model with the custom loss
model.compile(optimizer='adam', loss=custom_loss)

Train the Model:

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/50
827/827 [==============================] - 19s 13ms/step - loss: 2.0951 - val_loss: 1.9683
Epoch 2/50
827/827 [==============================] - 9s 11ms/step - loss: 1.9589 - val_loss: 1.9400
Epoch 3/50
827/827 [==============================] - 9s 11ms/step - loss: 1.8926 - val_loss: 1.8368
Epoch 4/50
827/827 [==============================] - 9s 11ms/step - loss: 1.8081 - val_loss: 1.7397
Epoch 5/50
827/827 [==============================] - 9s 11ms/step - loss: 1.7028 - val_loss: 1.6304
Epoch 6/50
827/827 [==============================] - 9s 11ms/step - loss: 1.5773 - val_loss: 1.5659
Epoch 7/50
827/827 [==============================] - 9s 11ms/step - loss: 1.4393 - val_loss: 1.3746
Epoch 8/50
827/827 [==============================] - 9s 11ms/step - loss: 1.3095 - val_loss: 1.3288
Epoch 9/50
827/827 [==============================] - 9s 11ms/step - loss: 1.1901 - val_loss: 1.2039
Epoch 10/50
827/827 [==============================] - 9s 11ms/step - loss: 1.0922 - val_l

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/LSTM_model_subset_3.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Make sure the custom_loss function is defined in the environment
model = load_model('/content/drive/MyDrive/Colab Notebooks/Python ML/Final project/LSTM_model_subset_3.keras', custom_objects={'custom_loss': custom_loss})



Model Evaluation:

In [ ]:
# Evaluate the model
test_loss = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")


414/414 [==============================] - 2s 4ms/step - loss: 1.9246
Test Loss: 1.9246209859848022


Seed seq

Genre List: afrobeat, afrocuban, blues, country, dance, funk, gospel, highlife, hiphop, jazz, latin, middleeastern, neworleans, pop, punk, reggae, rock, soul

In [ ]:
import numpy as np
import mido

def midi_to_seed_sequence(midi_file, sequence_length=50):
    """
    Converts a MIDI file to a seed sequence formatted like the training data.

    :param midi_file: Path to the MIDI file
    :param sequence_length: The length of the seed sequence
    :return: A seed sequence extracted from the MIDI file
    """
    midi = mido.MidiFile(midi_file)

    notes = []
    for track in midi.tracks:
        for msg in track:
            if not msg.is_meta and msg.type in ['note_on', 'note_off']:
                # Extract note number, velocity, and time
                note_data = [msg.note, msg.velocity, msg.time]
                notes.append(note_data)
                if len(notes) >= sequence_length:
                    break
        if len(notes) >= sequence_length:
            break

    # Pad with zeros if the sequence is shorter than desired
    if len(notes) < sequence_length:
        padding = sequence_length - len(notes)
        notes.extend([[0, 0, 0]] * padding)

    return np.array(notes)


afrobeat_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session3/11_afrobeat_94_beat_4-4.mid')#drummer1/session3/11_afrobeat_94_beat_4-4.mid
blues_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer4/session1/6_blues-shuffle_134_beat_4-4.mid')
country_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session2/11_country_114_fill_4-4.mid')#drummer1/session2/11_country_114_fill_4-4.mid
dance_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session1/101_dance-disco_120_beat_4-4.mid')#drummer1/session1/101_dance-disco_120_beat_4-4.mid
funk_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session1/122_funk_95_fill_4-4.mid')#drummer1/session1/122_funk_95_fill_4-4.mid
gospel_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session2/142_gospel_120_beat_4-4.mid')#drummer1/session2/142_gospel_120_beat_4-4.mid
hiphop_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid')#drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid
jazz_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/session1/10_jazz-funk_116_fill_4-4.mid')#drummer1/session1/10_jazz-funk_116_fill_4-4.mid
rock_seed_sequence = midi_to_seed_sequence('/content/drive/My Drive/Colab Notebooks/Python ML/Final project/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid')#drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid



In [ ]:
print(jazz_seed_sequence)

[[ 38  96   6]
 [ 38   0  12]
 [ 44  62  10]
 [ 44   0  17]
 [ 48  97  18]
 [ 46  13   9]
 [ 48   0   2]
 [ 46   0  15]
 [ 48 114  42]
 [ 48   0  14]
 [ 44  75  10]
 [ 44   0  17]
 [ 38  91   8]
 [ 38   0  15]
 [ 38  95   8]
 [ 38   0   6]
 [ 44  92   6]
 [ 44   0   7]
 [ 43  91   6]
 [ 43   0  12]
 [ 43  93  20]
 [ 43   0  27]
 [ 44  87   0]
 [ 44   0  18]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]]


In [ ]:
#############################################################################
seed_sequence = jazz_seed_sequence
#############################################################################

In [ ]:
max_time_value = max([note[2] for note in seed_sequence])
print(max_time_value)
# Assuming seed_sequence is an array with columns for note, velocity, and time
normalized_seed_sequence = np.stack([
    simple_normalize(seed_sequence[:, 0], 127),
    simple_normalize(seed_sequence[:, 1], 127),
    simple_normalize(seed_sequence[:, 2], max_time_value)  # Replace 'max_time_value' with the actual max
], axis=1)


print(normalized_seed_sequence)

42
[[0.2992126  0.75590551 0.14285714]
 [0.2992126  0.         0.28571429]
 [0.34645669 0.48818898 0.23809524]
 [0.34645669 0.         0.4047619 ]
 [0.37795276 0.76377953 0.42857143]
 [0.36220472 0.1023622  0.21428571]
 [0.37795276 0.         0.04761905]
 [0.36220472 0.         0.35714286]
 [0.37795276 0.8976378  1.        ]
 [0.37795276 0.         0.33333333]
 [0.34645669 0.59055118 0.23809524]
 [0.34645669 0.         0.4047619 ]
 [0.2992126  0.71653543 0.19047619]
 [0.2992126  0.         0.35714286]
 [0.2992126  0.7480315  0.19047619]
 [0.2992126  0.         0.14285714]
 [0.34645669 0.72440945 0.14285714]
 [0.34645669 0.         0.16666667]
 [0.33858268 0.71653543 0.14285714]
 [0.33858268 0.         0.28571429]
 [0.33858268 0.73228346 0.47619048]
 [0.33858268 0.         0.64285714]
 [0.34645669 0.68503937 0.        ]
 [0.34645669 0.         0.42857143]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0

In [ ]:
'''
# Assuming note_scaler, velocity_scaler, and time_scaler are already fitted to your training data
def normalize_seed_sequence(seed_sequence, note_scaler, velocity_scaler, time_scaler):
    normalized_sequence = np.array(seed_sequence)  # Copy to avoid modifying the original

    # Normalize each feature
    normalized_sequence[:, 0] = note_scaler.transform(seed_sequence[:, [0]]).flatten()  # Notes
    normalized_sequence[:, 1] = velocity_scaler.transform(seed_sequence[:, [1]]).flatten()  # Velocities
    normalized_sequence[:, 2] = time_scaler.transform(seed_sequence[:, [2]]).flatten()  # Time

    return normalized_sequence

note_scaler_seed = MinMaxScaler(feature_range=(0, 1)).fit(np.array(seed_sequence).reshape(-1, 1))
velocity_scaler_seed = MinMaxScaler(feature_range=(0, 1)).fit(np.array(seed_sequence).reshape(-1, 1))
time_scaler_seed = MinMaxScaler(feature_range=(0, 1)).fit(np.array(seed_sequence).reshape(-1, 1))

# Normalize the seed sequence
normalized_seed_sequence = normalize_seed_sequence(seed_sequence, note_scaler_seed, velocity_scaler_seed, time_scaler_seed)
'''

In [ ]:
import mido
import tensorflow_datasets as tfds
import io

# Load the dataset
dataset = tfds.load(name="groove/full-midionly", split="train", try_gcs=True)

# Function to extract TPQN from a MIDI file in the dataset
def get_tpqn_from_midi(midi_data):
    midi_stream = io.BytesIO(midi_data.numpy())
    midi_file = mido.MidiFile(file=midi_stream)
    return midi_file.ticks_per_beat

# Iterate through the dataset to find TPQN
for features in dataset.take(1):  # Just checking the first file for example
    midi_data = features['midi']
    tpqn = get_tpqn_from_midi(midi_data)
    print("Ticks Per Quarter Note (TPQN):", tpqn)
    break  # Only check the first file


Ticks Per Quarter Note (TPQN): 480


In [ ]:
def adjust_timing_to_heartbeat(continuous_data, heartbeat_bpm):
    # Adjust only the time component of the continuous data
    beat_duration_in_ticks = (60 / heartbeat_bpm)*480
    #beat_duration_in_ticks = heartbeat_bpm
    continuous_data[:, -1] = beat_duration_in_ticks  # Assuming last column is time
    return continuous_data

In [ ]:
def generate_heartbeat_synced_music(seed_sequence, heartbeat_bpm, num_steps, sequence_length):
    generated_sequence = seed_sequence
    current_sequence = seed_sequence

    for _ in range(num_steps):
        # Reshape current_sequence for prediction
        current_sequence_reshaped = current_sequence[-sequence_length:].reshape(1, sequence_length, 3)

        # Predict the next part of the sequence
        next_part = model.predict(current_sequence_reshaped)
        print(next_part)

        # Convert one-hot encoded notes to single note values
        next_notes = np.argmax(next_part[:, :22], axis=1)  # Get the index of the '1' in one-hot
        next_continuous = next_part[:, 22:]  # Velocity and time

        # Adjust the timing to match the heartbeat
        adjusted_continuous = adjust_timing_to_heartbeat(next_continuous, heartbeat_bpm)

        # Combine note with continuous features
        adjusted_next_part = np.column_stack([next_notes, adjusted_continuous])

        # Update the generated sequence
        generated_sequence = np.concatenate([generated_sequence, adjusted_next_part])

        # Update the current sequence for the next prediction
        current_sequence = np.concatenate([current_sequence, adjusted_next_part])

    return generated_sequence



# Example usage
#seed_sequence = select_seed_sequence()  # This is your chosen seed sequence
#heartbeat_bpm = get_current_heartbeat_bpm()  # Function to get the current heartbeat BPM
heartbeat_bpm = 80
generated_music = generate_heartbeat_synced_music(normalized_seed_sequence, heartbeat_bpm, num_steps=50, sequence_length = 50)


1/1 [==============================] - 0s 85ms/step
[[6.9540017e-04 3.0098408e-07 6.8483838e-05 2.1403558e-04 5.2602384e-02
  9.4641823e-01 1.0142899e-06 6.1329147e-10 6.7633863e-08 1.8326886e-14
  3.0800211e-09 2.5711624e-13 8.6267451e-15 1.2233693e-15 3.8377305e-15
  4.3673878e-15 7.9424794e-08 1.3298242e-13 1.6930587e-12 1.5741773e-09
  4.2305382e-12 1.1384031e-13 4.8390222e-01 2.2795860e-02]]
1/1 [==============================] - 0s 82ms/step
[[4.5652229e-01 7.7158340e-08 1.2228766e-06 9.9034846e-02 4.4095585e-01
  3.7323975e-04 3.1124661e-03 4.9212345e-09 1.7537742e-08 5.7242381e-16
  5.7998203e-11 1.3597328e-11 4.3725482e-16 1.0130587e-16 5.7345838e-13
  1.8650433e-15 3.8609973e-09 4.0343641e-15 1.4540656e-12 4.6525175e-10
  2.2179372e-09 8.9067729e-12 3.6927328e-01 3.4520060e-02]]
1/1 [==============================] - 0s 92ms/step
[[6.71221495e-01 6.77299909e-07 1.41704595e-02 4.12258987e-06
  3.15322503e-02 3.88378157e-06 2.83066958e-01 1.03507560e-07
  5.47507177e-08 2.91381

In [ ]:
print(generated_music.shape)

(100, 3)


In [ ]:
print(generated_music)

[[2.99212598e-01 7.55905512e-01 1.42857143e-01]
 [2.99212598e-01 0.00000000e+00 2.85714286e-01]
 [3.46456693e-01 4.88188976e-01 2.38095238e-01]
 [3.46456693e-01 0.00000000e+00 4.04761905e-01]
 [3.77952756e-01 7.63779528e-01 4.28571429e-01]
 [3.62204724e-01 1.02362205e-01 2.14285714e-01]
 [3.77952756e-01 0.00000000e+00 4.76190476e-02]
 [3.62204724e-01 0.00000000e+00 3.57142857e-01]
 [3.77952756e-01 8.97637795e-01 1.00000000e+00]
 [3.77952756e-01 0.00000000e+00 3.33333333e-01]
 [3.46456693e-01 5.90551181e-01 2.38095238e-01]
 [3.46456693e-01 0.00000000e+00 4.04761905e-01]
 [2.99212598e-01 7.16535433e-01 1.90476190e-01]
 [2.99212598e-01 0.00000000e+00 3.57142857e-01]
 [2.99212598e-01 7.48031496e-01 1.90476190e-01]
 [2.99212598e-01 0.00000000e+00 1.42857143e-01]
 [3.46456693e-01 7.24409449e-01 1.42857143e-01]
 [3.46456693e-01 0.00000000e+00 1.66666667e-01]
 [3.38582677e-01 7.16535433e-01 1.42857143e-01]
 [3.38582677e-01 0.00000000e+00 2.85714286e-01]
 [3.38582677e-01 7.32283465e-01 4.761904

generated seq->midi

In [ ]:
'''
import mido
from mido import MidiFile, MidiTrack, Message
def sequence_to_midi(generated_sequence, note_scaler, velocity_scaler, time_scaler, output_file='generated_music.mid'):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Initial settings
    track.append(Message('program_change', program=12, time=0))

    last_time = 0
    for note_data in generated_sequence:
        # Denormalize the note, velocity, and time
        note = int(note_scaler.inverse_transform([[note_data[0]]])[0][0])
        velocity = int(velocity_scaler.inverse_transform([[note_data[1]]])[0][0])
        time = time_scaler.inverse_transform([[note_data[2]]])[0][0]

        # Clamp the note and velocity to MIDI range 0-127
        note = max(0, min(127, note))
        velocity = max(0, min(127, velocity))

        #print(note)
        #print(velocity)

        # Convert time to integer as MIDI ticks
        midi_time = int(time * mid.ticks_per_beat)

        # Calculate time delta
        time_delta = midi_time - last_time
        if time_delta < 0:
            time_delta = 0  # Ensure non-negative time

        if velocity > 0:  # note_on
            track.append(Message('note_on', note=note, velocity=velocity, time=time_delta))
        else:  # note_off
            track.append(Message('note_off', note=note, velocity=0, time=time_delta))

        last_time = midi_time

    mid.save(output_file)



# Convert your generated music to MIDI
sequence_to_midi(generated_music, note_scaler, velocity_scaler, time_scaler, output_file='/content/drive/My Drive/Colab Notebooks/Python ML/Final project/generated_music.mid')
'''


"\nimport mido\nfrom mido import MidiFile, MidiTrack, Message\ndef sequence_to_midi(generated_sequence, note_scaler, velocity_scaler, time_scaler, output_file='generated_music.mid'):\n    mid = MidiFile()\n    track = MidiTrack()\n    mid.tracks.append(track)\n\n    # Initial settings\n    track.append(Message('program_change', program=12, time=0))\n\n    last_time = 0\n    for note_data in generated_sequence:\n        # Denormalize the note, velocity, and time\n        note = int(note_scaler.inverse_transform([[note_data[0]]])[0][0])\n        velocity = int(velocity_scaler.inverse_transform([[note_data[1]]])[0][0])\n        time = time_scaler.inverse_transform([[note_data[2]]])[0][0]\n\n        # Clamp the note and velocity to MIDI range 0-127\n        note = max(0, min(127, note))\n        velocity = max(0, min(127, velocity))\n\n        #print(note)\n        #print(velocity)\n\n        # Convert time to integer as MIDI ticks\n        midi_time = int(time * mid.ticks_per_beat)\n\n   

In [ ]:
'''
def manual_inverse_normalize(scaled_data, data_min, data_max):
    return scaled_data * (data_max - data_min) + data_min
'''

In [ ]:
def simple_denormalize(data, max_value):
    return (data * max_value).astype(int)

def denormalize_generated_music(generated_sequence, max_time_value):
    denormalized_music = []

    for sequence in generated_sequence:
        # Denormalizing notes and velocities to integers
        note = simple_denormalize(sequence[0], 127)
        velocity = simple_denormalize(sequence[1], 127)

        note = max(0, min(127, note))
        velocity = max(0, min(127, velocity))

        # Time might also need to be an integer depending on how it's used in MIDI
        time = simple_denormalize(sequence[2], max_time_value)

        denormalized_music.append([note, velocity, time])

    return np.array(denormalized_music)


In [ ]:

print(max_time_value)
denormalized_music = denormalize_generated_music(generated_music, max_time_value)

42


In [ ]:
print(denormalized_music)

[[  38   96    6]
 [  38    0   12]
 [  44   62   10]
 [  44    0   17]
 [  48   97   18]
 [  46   13    9]
 [  48    0    2]
 [  46    0   15]
 [  48  114   42]
 [  48    0   14]
 [  44   75   10]
 [  44    0   17]
 [  38   91    8]
 [  38    0   15]
 [  38   95    8]
 [  38    0    6]
 [  44   92    6]
 [  44    0    7]
 [  43   91    6]
 [  43    0   12]
 [  43   93   20]
 [  43    0   27]
 [  44   87    0]
 [  44    0   18]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [   0    0    0]
 [ 127   61 3360]
 [   0   46 3360]
 [   0   49 3360]
 [ 127   59 3360]
 [ 127   64 3360]
 [ 127   6

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message

def sequence_to_midi(denormalized_sequence, output_file='generated_music.mid'):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(Message('program_change', program=12, time=0))

    last_time = 0
    for note_data in denormalized_sequence:
        note, velocity, time = note_data

        # Convert time to integer as MIDI ticks
        midi_time = int(time * mid.ticks_per_beat)

        # Calculate time delta
        time_delta = midi_time - last_time
        if time_delta < 0:
            time_delta = 0

        # Create note_on or note_off message
        if velocity > 0:
            track.append(Message('note_on', note=note, velocity=velocity, time=time_delta))
        else:
            track.append(Message('note_off', note=note, velocity=0, time=time_delta))

        last_time = midi_time

    mid.save(output_file)


In [ ]:
# Create a MIDI file from the denormalized music
sequence_to_midi(denormalized_music, '/content/drive/My Drive/Colab Notebooks/Python ML/Final project/generated_music.mid')
